In [17]:

%load_ext autoreload
%autoreload 2

import pandas as pd

pd.options.display.max_colwidth = 1000
#df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import pandas as pd
from glob import glob

files = glob("../data/test*beto*.csv")

dfs = [pd.read_csv(f) for f in files]


df_test = pd.concat(dfs)

df_test.shape


(5670, 23)

## Error analysis

In [19]:
from rioplatense_hs.preprocessing import text_to_label, labels
from sklearn.metrics import classification_report

pred_labels = [f"PRED_{label}" for label in labels]
gold_labels = df_test[labels]
preds = (df_test[pred_labels] > 0.5).astype(int)


print(classification_report(gold_labels, preds, target_names=labels))

              precision    recall  f1-score   support

       WOMEN       0.58      0.34      0.43       131
       LGBTI       0.59      0.38      0.46        88
      RACISM       0.80      0.70      0.75       230
       CLASS       0.73      0.47      0.58        76
    POLITICS       0.76      0.49      0.60       144
    DISABLED       0.63      0.58      0.60        66
  APPEARANCE       0.81      0.72      0.76       189
    CRIMINAL       0.78      0.65      0.71       185

   micro avg       0.75      0.58      0.65      1109
   macro avg       0.71      0.54      0.61      1109
weighted avg       0.74      0.58      0.64      1109
 samples avg       0.10      0.09      0.10      1109



/users/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/users/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
pred_labels = [l for l in df_test.columns if l.startswith("PRED")]

label_hate = df_test[labels].sum(axis=1) > 0
pred_hate = (df_test[pred_labels] > 0.5).sum(axis=1) > 0

print(classification_report(label_hate, pred_hate))

              precision    recall  f1-score   support

       False       0.94      0.96      0.95      4794
        True       0.77      0.67      0.72       876

    accuracy                           0.92      5670
   macro avg       0.86      0.82      0.84      5670
weighted avg       0.92      0.92      0.92      5670



In [22]:
df_test.to_csv("../data/test_beto.csv", index=False)